# Mini Projet - Extraction et Analyse des données géographiques du Burkina Faso

Ce notebook réalise l'extraction et l'analyse des données géographiques du Burkina Faso depuis la base de données GeoNames.

**Auteur:** SAWADOGO Abdel Saïd Najib - Étudiant en Fouilles de données et intelligence artificielle  
**Date:** 15 Août 2025

---

## 📚 Import des bibliothèques

In [1]:
import pandas as pd
import numpy as np
import openpyxl
import zipfile
import requests
import os
from io import StringIO
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("MINI PROJET - ANALYSE DES DONNÉES GÉOGRAPHIQUES DU BURKINA FASO")
print("="*80)

MINI PROJET - ANALYSE DES DONNÉES GÉOGRAPHIQUES DU BURKINA FASO


## ⚙️ Configuration des paramètres

In [2]:
# Configuration des URLs et noms de fichiers
GEONAMES_URL = "https://download.geonames.org/export/dump/BF.zip"
ZIP_FILENAME = "BF.zip"
TXT_FILENAME = "BF.txt"
OUTPUT_CSV = "burkina_location.csv"
GOUNGHIN_CSV = "gounghin.csv"
EXCEL_FILENAME = "mini_projet.xlsx"

print("Configuration terminée ✅")

Configuration terminée ✅


## 🔄 ÉTAPE 1: Téléchargement des données du Burkina Faso

In [3]:
def download_burkina_data():
    """Télécharge le fichier ZIP des données du Burkina Faso"""
    try:
        print(f"📥 Téléchargement depuis: {GEONAMES_URL}")
        response = requests.get(GEONAMES_URL, stream=True)
        response.raise_for_status()
        
        with open(ZIP_FILENAME, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"✅ Téléchargement terminé: {ZIP_FILENAME}")
        return True
    except Exception as e:
        print(f"❌ Erreur lors du téléchargement: {e}")
        return False

In [4]:
# Exécuter le téléchargement
print("-" * 50)
if download_burkina_data():
    print(f"📁 Taille du fichier téléchargé: {os.path.getsize(ZIP_FILENAME) / 1024:.1f} KB")
else:
    print("❌ Échec du téléchargement")

--------------------------------------------------
📥 Téléchargement depuis: https://download.geonames.org/export/dump/BF.zip
✅ Téléchargement terminé: BF.zip
📁 Taille du fichier téléchargé: 307.2 KB


## 🔄 ÉTAPE 2: Extraction et prétraitement des données

In [5]:
def extract_and_preprocess():
    """Extrait le fichier ZIP et prétraite les données"""
    try:
        # Extraction du fichier ZIP
        with zipfile.ZipFile(ZIP_FILENAME, 'r') as zip_ref:
            zip_ref.extractall()
            print(f"✅ Fichier extrait: {TXT_FILENAME}")
        
        # Définition des colonnes selon la documentation GeoNames
        columns = [
            'geonameid', 'name', 'asciiname', 'alternatenames',
            'latitude', 'longitude', 'feature_class', 'feature_code',
            'country_code', 'cc2', 'admin1_code', 'admin2_code',
            'admin3_code', 'admin4_code', 'population', 'elevation',
            'dem', 'timezone', 'modification_date'
        ]
        
        # Lecture des données
        print("📊 Chargement des données...")
        df = pd.read_csv(TXT_FILENAME, sep='\t', names=columns, encoding='utf-8')
        print(f"✅ Données chargées: {len(df)} enregistrements")
        
        # Sélection et renommage des colonnes requises
        df_filtered = df[['geonameid', 'name', 'latitude', 'longitude']].copy()
        df_filtered = df_filtered.rename(columns={
            'geonameid': 'ID',
            'name': 'location_name',
            'latitude': 'lat',
            'longitude': 'long'
        })
        
        # Nettoyage des données
        df_filtered = df_filtered.dropna()
        df_filtered['lat'] = pd.to_numeric(df_filtered['lat'], errors='coerce')
        df_filtered['long'] = pd.to_numeric(df_filtered['long'], errors='coerce')
        df_filtered = df_filtered.dropna()
        
        # Sauvegarde en CSV
        df_filtered.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')
        print(f"✅ Données sauvegardées: {OUTPUT_CSV}")
        print(f"📈 Nombre d'enregistrements traités: {len(df_filtered)}")
        
        return df_filtered
        
    except Exception as e:
        print(f"❌ Erreur lors de l'extraction: {e}")
        return None

In [6]:
# Exécuter l'extraction et le prétraitement
print("-" * 50)
burkina_data = extract_and_preprocess()

--------------------------------------------------
✅ Fichier extrait: BF.txt
📊 Chargement des données...
✅ Données chargées: 11958 enregistrements
✅ Données sauvegardées: burkina_location.csv
📈 Nombre d'enregistrements traités: 11958


## 📊 Aperçu des données

In [7]:
if burkina_data is not None:
    # Affichage d'un aperçu des données
    print("📋 Aperçu des données:")
    display(burkina_data.head(10))
    
    print(f"\nℹ️  Statistiques de base:")
    print(f"   - Nombre total de lieux: {len(burkina_data)}")
    print(f"   - Latitude min: {burkina_data['lat'].min():.6f}")
    print(f"   - Latitude max: {burkina_data['lat'].max():.6f}")
    print(f"   - Longitude min: {burkina_data['long'].min():.6f}")
    print(f"   - Longitude max: {burkina_data['long'].max():.6f}")
else:
    print("❌ Aucune donnée à afficher")

📋 Aperçu des données:


,ID,location_name,lat,long
0,2282318,Pouéné,9.72908,-2.78660
1,2285251,Léraba Occidentale,10.28333,-5.11667
2,2287216,Kéléouoro,9.80748,-4.05023
3,2294066,White Volta,8.70194,-0.99056
4,2298457,Issana Bouga,10.91667,-1.18333
5,2353158,Zyonguen,12.36667,-0.45000
6,2353159,Zyiliwèlè,12.38333,-2.73333
7,2353160,Zyanko,12.78333,-0.41667
8,2353161,Zouta,13.14908,-1.28197
9,2353162,Zourtenga,12.95741,-1.28745



ℹ️  Statistiques de base:
   - Nombre total de lieux: 11958
   - Latitude min: 5.216090
   - Latitude max: 15.077670
   - Longitude min: -5.659680
   - Longitude max: 2.522590


## 🔄 ÉTAPE 4: Opérations d'analyse sur les données

In [8]:
def analyze_burkina_data(df):
    """Effectue les analyses demandées"""
    results = {}
    
    # 4.1 - Extraction des données contenant 'gounghin'
    print("🔍 4.1 - Recherche des lieux contenant 'gounghin'...")
    gounghin_data = df[df['location_name'].str.contains('gounghin', case=False, na=False)]
    gounghin_data.to_csv(GOUNGHIN_CSV, index=False, encoding='utf-8')
    print(f"✅ Trouvé {len(gounghin_data)} lieu(x) avec 'gounghin'")
    print(f"💾 Sauvegardé dans: {GOUNGHIN_CSV}")
    if not gounghin_data.empty:
        print("   Lieux trouvés:")
        for _, row in gounghin_data.iterrows():
            print(f"   - {row['location_name']} (ID: {row['ID']}, Lat: {row['lat']:.6f}, Long: {row['long']:.6f})")
    results['gounghin'] = gounghin_data
    
    return results

### 4.1 - Recherche des lieux contenant 'gounghin'

In [9]:
if burkina_data is not None:
    print("-" * 50)
    # 4.1 - Extraction des données contenant 'gounghin'
    print("🔍 4.1 - Recherche des lieux contenant 'gounghin'...")
    gounghin_data = burkina_data[burkina_data['location_name'].str.contains('gounghin', case=False, na=False)]
    gounghin_data.to_csv(GOUNGHIN_CSV, index=False, encoding='utf-8')
    print(f"✅ Trouvé {len(gounghin_data)} lieu(x) avec 'gounghin'")
    print(f"💾 Sauvegardé dans: {GOUNGHIN_CSV}")
    
    if not gounghin_data.empty:
        print("\n📋 Lieux trouvés:")
        display(gounghin_data)
    else:
        print("❌ Aucun lieu trouvé contenant 'gounghin'")

--------------------------------------------------
🔍 4.1 - Recherche des lieux contenant 'gounghin'...
✅ Trouvé 10 lieu(x) avec 'gounghin'
💾 Sauvegardé dans: gounghin.csv

📋 Lieux trouvés:


,ID,location_name,lat,long
153,2353306,Gounghin,12.06677,-1.42134
7269,2360473,Gounghin,12.62488,-1.36398
10260,2570204,Gounghin,12.31436,-1.37900
10746,10342749,Gounghin,12.06667,-0.15000
10759,10629032,BICIAB // Gounghin,12.35921,-1.54273
10818,11257296,Gounghin Department,12.06671,-0.15484
10845,11900526,Gounghin Nord,12.36120,-1.55055
10846,11900528,Zone Industrielle de Gounghin,12.36631,-1.54137
10852,11900619,Gounghin Sud,12.35298,-1.54342
10866,11900680,Gounghin,12.35895,-1.54442


### 4.2 - Extraction des lieux A-P

In [10]:
if burkina_data is not None:
    print("🔍 4.2 - Extraction des lieux dont la première lettre est entre A et P...")
    a_to_p_data = burkina_data[burkina_data['location_name'].str[0].str.upper().between('A', 'P', inclusive='both')]
    print(f"✅ Trouvé {len(a_to_p_data)} lieu(x) commençant par A-P")
    
    print("\n📋 Aperçu des premiers lieux A-P:")
    display(a_to_p_data.head(10))

🔍 4.2 - Extraction des lieux dont la première lettre est entre A et P...
✅ Trouvé 8306 lieu(x) commençant par A-P

📋 Aperçu des premiers lieux A-P:


,ID,location_name,lat,long
0,2282318,Pouéné,9.72908,-2.78660
1,2285251,Léraba Occidentale,10.28333,-5.11667
2,2287216,Kéléouoro,9.80748,-4.05023
4,2298457,Issana Bouga,10.91667,-1.18333
137,2353290,Forêt Classée de Ziga,12.47106,-1.08644
153,2353306,Gounghin,12.06677,-1.42134
320,2353473,Dar Salam,12.36146,-1.63909
389,2353543,Forêt Classée de Yendéré,10.15000,-5.06667
414,2353568,Province du Yatenga,13.58333,-2.41667
558,2353713,Forêt Classée de Yabo,12.98801,-1.50561


### 4.3 - Coordonnées extrêmes

In [11]:
if burkina_data is not None:
    print("🔍 4.3 - Recherche des coordonnées extrêmes...")
    
    lat_min_idx = burkina_data['lat'].idxmin()
    lat_max_idx = burkina_data['lat'].idxmax()
    long_min_idx = burkina_data['long'].idxmin()
    long_max_idx = burkina_data['long'].idxmax()
    
    print("📊 Coordonnées extrêmes:")
    print(f"   Latitude minimale: {burkina_data.loc[lat_min_idx, 'lat']:.6f} - {burkina_data.loc[lat_min_idx, 'location_name']}")
    print(f"   Latitude maximale: {burkina_data.loc[lat_max_idx, 'lat']:.6f} - {burkina_data.loc[lat_max_idx, 'location_name']}")
    print(f"   Longitude minimale: {burkina_data.loc[long_min_idx, 'long']:.6f} - {burkina_data.loc[long_min_idx, 'location_name']}")
    print(f"   Longitude maximale: {burkina_data.loc[long_max_idx, 'long']:.6f} - {burkina_data.loc[long_max_idx, 'location_name']}")
    
    # Créer un DataFrame avec les coordonnées extrêmes
    extremes_data = pd.DataFrame([
        burkina_data.loc[lat_min_idx],
        burkina_data.loc[lat_max_idx],
        burkina_data.loc[long_min_idx],
        burkina_data.loc[long_max_idx]
    ], index=['Latitude Min', 'Latitude Max', 'Longitude Min', 'Longitude Max'])
    
    print("\n📋 Tableau des coordonnées extrêmes:")
    display(extremes_data)

🔍 4.3 - Recherche des coordonnées extrêmes...
📊 Coordonnées extrêmes:
   Latitude minimale: 5.216090 - Komoé
   Latitude maximale: 15.077670 - Lalaba
   Longitude minimale: -5.659680 - Banifing
   Longitude maximale: 2.522590 - Tapoa

📋 Tableau des coordonnées extrêmes:


,ID,location_name,lat,long
Latitude Min,2359210,Komoé,5.21609,-3.71793
Latitude Max,6913777,Lalaba,15.07767,-0.49986
Longitude Min,2357400,Banifing,12.01147,-5.65968
Longitude Max,2439128,Tapoa,12.54850,2.52259


### 4.4 - Lieux avec coordonnées spécifiques (lat >= 11 et lon <= 0.5)

In [12]:
if burkina_data is not None:
    print("🔍 4.4 - Lieux avec lat >= 11 et lon <= 0.5...")
    specific_coords = burkina_data[(burkina_data['lat'] >= 11) & (burkina_data['long'] <= 0.5)]
    print(f"✅ Trouvé {len(specific_coords)} lieu(x) avec lat >= 11 et long <= 0.5")
    
    if not specific_coords.empty:
        print("\n📋 Premiers lieux avec ces coordonnées:")
        display(specific_coords.head(10))
    else:
        print("❌ Aucun lieu trouvé avec ces coordonnées")

🔍 4.4 - Lieux avec lat >= 11 et lon <= 0.5...
✅ Trouvé 9466 lieu(x) avec lat >= 11 et long <= 0.5

📋 Premiers lieux avec ces coordonnées:


,ID,location_name,lat,long
5,2353158,Zyonguen,12.36667,-0.45000
6,2353159,Zyiliwèlè,12.38333,-2.73333
7,2353160,Zyanko,12.78333,-0.41667
8,2353161,Zouta,13.14908,-1.28197
9,2353162,Zourtenga,12.95741,-1.28745
10,2353163,Zourma-Kita,11.39418,-0.79273
11,2353164,Zouri,11.65000,-1.28333
12,2353165,Zoura,13.28261,-1.47644
13,2353166,Zoungwa,13.21895,-1.97142
14,2353167,Zoungou,12.14441,-0.53169


## 🔄 ÉTAPE 5: Création du fichier Excel

In [13]:
def create_excel_file(burkina_data):
    """Crée le fichier Excel avec les différentes feuilles"""
    try:
        # Recalculer les analyses pour le fichier Excel
        gounghin_data = burkina_data[burkina_data['location_name'].str.contains('gounghin', case=False, na=False)]
        a_to_p_data = burkina_data[burkina_data['location_name'].str[0].str.upper().between('A', 'P', inclusive='both')]
        specific_coords = burkina_data[(burkina_data['lat'] >= 11) & (burkina_data['long'] <= 0.5)]
        
        # Coordonnées extrêmes
        lat_min_idx = burkina_data['lat'].idxmin()
        lat_max_idx = burkina_data['lat'].idxmax()
        long_min_idx = burkina_data['long'].idxmin()
        long_max_idx = burkina_data['long'].idxmax()
        
        extremes = {
            'lat_min': burkina_data.loc[lat_min_idx],
            'lat_max': burkina_data.loc[lat_max_idx],
            'long_min': burkina_data.loc[long_min_idx],
            'long_max': burkina_data.loc[long_max_idx]
        }
        
        with pd.ExcelWriter(EXCEL_FILENAME, engine='openpyxl') as writer:
            # Feuille 1: données gounghin
            if not gounghin_data.empty:
                gounghin_data.to_excel(writer, sheet_name='gounghin', index=False)
                print(f"✅ Feuille 'gounghin' créée avec {len(gounghin_data)} enregistrements")
            else:
                # Créer une feuille vide si aucune donnée
                pd.DataFrame(columns=['ID', 'location_name', 'lat', 'long']).to_excel(
                    writer, sheet_name='gounghin', index=False)
                print("✅ Feuille 'gounghin' créée (vide)")
            
            # Feuille 2: données A-P
            a_to_p_data.to_excel(writer, sheet_name='A_to_P', index=False)
            print(f"✅ Feuille 'A_to_P' créée avec {len(a_to_p_data)} enregistrements")
            
            # Feuille 3: Résumé des analyses
            summary_data = []
            summary_data.append(['Analyse', 'Résultat'])
            summary_data.append(['Nombre total de lieux', len(burkina_data)])
            summary_data.append(['Lieux avec "gounghin"', len(gounghin_data)])
            summary_data.append(['Lieux A-P', len(a_to_p_data)])
            summary_data.append(['Lieux lat>=11 et long<=0.5', len(specific_coords)])
            summary_data.append(['', ''])
            summary_data.append(['Coordonnées extrêmes', ''])
            summary_data.append(['Latitude min', f"{extremes['lat_min']['lat']:.6f} - {extremes['lat_min']['location_name']}"])
            summary_data.append(['Latitude max', f"{extremes['lat_max']['lat']:.6f} - {extremes['lat_max']['location_name']}"])
            summary_data.append(['Longitude min', f"{extremes['long_min']['long']:.6f} - {extremes['long_min']['location_name']}"])
            summary_data.append(['Longitude max', f"{extremes['long_max']['long']:.6f} - {extremes['long_max']['location_name']}"])
            
            summary_df = pd.DataFrame(summary_data)
            summary_df.to_excel(writer, sheet_name='Résumé', index=False, header=False)
            print("✅ Feuille 'Résumé' créée avec les statistiques")
        
        print(f"📊 Fichier Excel créé: {EXCEL_FILENAME}")
        return True
        
    except Exception as e:
        print(f"❌ Erreur lors de la création du fichier Excel: {e}")
        return False

In [14]:
# Créer le fichier Excel
if burkina_data is not None:
    print("-" * 50)
    create_excel_file(burkina_data)
else:
    print("❌ Impossible de créer le fichier Excel - données manquantes")

--------------------------------------------------
✅ Feuille 'gounghin' créée avec 10 enregistrements
✅ Feuille 'A_to_P' créée avec 8306 enregistrements
✅ Feuille 'Résumé' créée avec les statistiques
📊 Fichier Excel créé: mini_projet.xlsx


## ✅ Résumé final

In [15]:
if burkina_data is not None:
    # Recalcul des statistiques pour le résumé
    gounghin_count = len(burkina_data[burkina_data['location_name'].str.contains('gounghin', case=False, na=False)])
    a_to_p_count = len(burkina_data[burkina_data['location_name'].str[0].str.upper().between('A', 'P', inclusive='both')])
    specific_coords_count = len(burkina_data[(burkina_data['lat'] >= 11) & (burkina_data['long'] <= 0.5)])
    
    print("="*80)
    print("✅ TRAITEMENT TERMINÉ AVEC SUCCÈS!")
    print("="*80)
    print(f"📁 Fichiers créés:")
    print(f"   • {OUTPUT_CSV} - Données principales du Burkina Faso")
    print(f"   • {GOUNGHIN_CSV} - Lieux contenant 'gounghin'")
    print(f"   • {EXCEL_FILENAME} - Fichier Excel avec toutes les analyses")
    print(f"\n📊 Résumé des analyses:")
    print(f"   • Nombre total de lieux traités: {len(burkina_data)}")
    print(f"   • Lieux contenant 'gounghin': {gounghin_count}")
    print(f"   • Lieux commençant par A-P: {a_to_p_count}")
    print(f"   • Lieux avec lat>=11 et long<=0.5: {specific_coords_count}")
else:
    print("❌ Échec du traitement des données")

✅ TRAITEMENT TERMINÉ AVEC SUCCÈS!
📁 Fichiers créés:
   • burkina_location.csv - Données principales du Burkina Faso
   • gounghin.csv - Lieux contenant 'gounghin'
   • mini_projet.xlsx - Fichier Excel avec toutes les analyses

📊 Résumé des analyses:
   • Nombre total de lieux traités: 11958
   • Lieux contenant 'gounghin': 10
   • Lieux commençant par A-P: 8306
   • Lieux avec lat>=11 et long<=0.5: 9466
